## Test demo 

#### Parameters

In [2]:
dataset_folder = "D:\\Program\\Apollo\\instance"
sample_amount = 10

#### Build Sample dataset (only once)

In [ ]:
import json
from pycocotools.coco import COCO
import random

path = dataset_folder + "/annotations/apollo_keypoints_66_val.json"

with open(path,'r') as annotation_file:
    template = json.load(annotation_file)

coco = COCO(path)
image_ids = coco.getImgIds()
selected_image_ids = random.sample(image_ids,k=sample_amount)
selected_ann_id = coco.getAnnIds(imgIds=selected_image_ids)
template['images'] = coco.loadImgs(ids=selected_image_ids)
template['annotations'] = coco.loadAnns(ids=selected_ann_id)

with open("./sample/apollo_sample.json",'w') as new_file:
    json.dump(template,new_file)


#### Setup Dataset

In [3]:
import json
from detectron2.data import MetadataCatalog
from detectron2.data.datasets import register_coco_instances

register_coco_instances("apollo_sample", {}, "./sample/apollo_sample.json", dataset_folder + "/train/images")

with open("./configs/apollo_meta.json",'r') as load_f:
    apollo_metadata = json.load(load_f)

MetadataCatalog.get("apollo_sample").keypoint_names = apollo_metadata['APOLLO_CAR_KEYPOINT_NAMES']
MetadataCatalog.get("apollo_sample").keypoint_flip_map = apollo_metadata['APOLLO_CAR_KEYPOINT_FLIP_MAP']
MetadataCatalog.get("apollo_sample").keypoint_connection_rules = apollo_metadata['APOLLO_CAR_KEYPOINT_CONNECTION_RULES']
MetadataCatalog.get("apollo_sample").thing_classes = ['car']

#### Setup dataloader and predictor

In [4]:
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import build_detection_train_loader
from detectron2.engine import DefaultPredictor
import torch

cfg = get_cfg()
cfg.merge_from_file("./configs/Apollo-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml") 
cfg.MODEL.WEIGHTS = './models/box_keypoint.pth'
cfg.DATASETS.TRAIN = 'apollo_sample'
cfg.SOLVER.IMS_PER_BATCH = 1
data_loader = build_detection_train_loader(cfg,mapper=None)
predictor = DefaultPredictor(cfg)

In [15]:
import cv2
from detectron2.data import detection_utils as utils

# create a iterator from data loader
iterator = iter(data_loader)

for i in range(sample_amount):
    # get batch from data loader iterator
    batch = next(iterator)

    # get the only data pair from batch
    sample = batch[0]    

    # get the image from sample and convert from tensor to numpy
    numpy_img = sample["image"].permute(1, 2, 0).cpu().detach().numpy()
    numpy_img = utils.convert_image_to_rgb(numpy_img, cfg.INPUT.FORMAT)

    visualizer_pred = Visualizer(numpy_img)
    visualizer_gt = Visualizer(numpy_img)

    predictions = predictor(numpy_img)
    instances = predictions["instances"].to(torch.device("cpu"))
    instances = instances[instances.scores > 0.9]
    output = visualizer_pred.draw_instance_predictions(predictions=instances)

    target_fields = sample["instances"].get_fields()
    labels = [MetadataCatalog.get("apollo_sample").thing_classes[i] for i in target_fields["gt_classes"]]
    ground_truth = visualizer_gt.overlay_instances(
     labels=labels,
     boxes=target_fields.get("gt_boxes", None),
     masks=target_fields.get("gt_masks", None),
     keypoints=target_fields.get("gt_keypoints", None),
    )

    output.save(f"./sample/prediction_{i}.jpg")
    ground_truth.save(f"./sample/ground_truth_{i}.jpg")

        

#### Fetch one instance from dataset and draw annotation

In [6]:
import numpy as np
import copy
import cv2
from PIL import Image
from detectron2.data import detection_utils as utils

iterator = iter(data_loader)
batch = next(iterator)
batch = next(iterator)
batch = next(iterator)
batch = next(iterator)
batch = next(iterator)
batch = next(iterator)
batch = next(iterator)
sample = batch[0]

# get the image from sample and convert from tensor to numpy
numpy_img = sample["image"].permute(1, 2, 0).cpu().detach().numpy()
numpy_img = utils.convert_image_to_rgb(numpy_img, cfg.INPUT.FORMAT)

predictions = predictor(numpy_img)
instances = predictions["instances"].to(torch.device("cpu"))
instances = instances[instances.scores > 0.9]

target_fields = sample["instances"].get_fields()

gt_kpts = np.asarray(target_fields.get("gt_keypoints", None).tensor)[0]
pred_kpts = instances[0].pred_keypoints[0].numpy()

pred_img = copy.deepcopy(numpy_img)
pred_img = np.ascontiguousarray(pred_img, dtype=np.uint8)
ground_truth_img = copy.deepcopy(numpy_img)
ground_truth_img = np.ascontiguousarray(ground_truth_img, dtype=np.uint8)


for i, kpts in enumerate(pred_kpts):
    x,y,v = kpts
    if(v>2):
        continue
    cv2.circle(pred_img,(round(x),round(y)),3,(255,0,0),-1)
    cv2.putText(pred_img,str(i),(round(x),round(y)),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255, 0, 0),2)
im = Image.fromarray(pred_img)
im.save("./sample/pred.jpg")

for i,kpts in enumerate(gt_kpts):
    x,y,v = kpts
    cv2.circle(ground_truth_img,(round(x),round(y)),3,(255,0,0),-1)
    cv2.putText(ground_truth_img,str(i),(round(x),round(y)),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255, 0, 0),2)

im = Image.fromarray(ground_truth_img)
im.save("./sample/gt.jpg")

c:\Users\JuJuBear\.conda\envs\Hiwi\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [7]:
for i, kpts in enumerate(pred_kpts):
    print(kpts)

[ 580.1472    1075.2695       1.6325674]
[ 574.15857  1084.2606      1.150612]
[ 608.0944   1070.2744      9.915062]
[ 602.1057    1080.2645       2.4980485]
[ 606.09814  1093.2517      4.799906]
[ 604.1019    1101.2439       2.3749945]
[ 615.0812    1108.2369       1.9982246]
[ 623.0661   1097.2478      9.154888]
[ 619.0736   1024.3198     13.270606]
[ 626.0604   1051.2932      3.822943]
[ 631.051    1094.2507      5.592505]
[ 630.05286  1020.3238     11.292413]
[ 640.03406   1042.3021       1.6330662]
[ 642.0303   1051.2932      1.895093]
[ 645.0246    1049.2952       2.5750272]
[ 648.0189    1086.2587       2.7357628]
[ 653.0095  1035.309      2.31426]
[ 656.00385   1044.3002       1.8486983]
[ 658.99817   1042.3021       1.6340319]
[ 655.00574   1082.2626       1.8432146]
[ 662.9906    1078.2666       3.9941168]
[ 666.9831    1074.2705       5.4731197]
[ 662.9906    1039.305        1.0759795]
[6.6199249e+02 1.0413031e+03 4.0366828e-01]
[ 546.21136  1027.3169     17.410044]
[5.02294

In [8]:
for i, kpts in enumerate(gt_kpts):
    print(kpts)

[ 578.809  1075.2177    2.    ]
[ 573.77783 1084.1458     2.     ]
[ 609.3117 1070.679     2.    ]
[ 607.3634 1077.6531    2.    ]
[ 603.1459 1094.0425    2.    ]
[ 602.1717 1103.131     2.    ]
[ 613.1031 1107.0055    2.    ]
[ 621.6213 1097.5406    2.    ]
[0. 0. 0.]
[ 624.23926 1048.4447     2.     ]
[ 630.1394 1094.9723    2.    ]
[ 629.918 1018.107    2.   ]
[ 640.7885 1039.1956    2.    ]
[ 641.82904 1053.0996     2.     ]
[ 646.5614 1050.1716    2.    ]
[ 643.9323 1086.8579    2.    ]
[ 650.3251 1033.5443    2.    ]
[ 655.33966 1043.9115     2.     ]
[ 658.69934 1041.7196     2.     ]
[ 653.66815 1081.4502     2.     ]
[ 662.1863 1078.8763    2.    ]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
[ 616.9387 1024.4336    2.

In [ ]:
python -m openpifpaf.predict Flood_img_demo.jpg --checkpoint=shufflenetv2k16-apollo-66 -o images --instance-threshold 0.05 --seed-threshold 0.05 --line-width 4 --font-size 0

In [ ]:
python -m openpifpaf.predict images/peterbourg.jpg --checkpoint=shufflenetv2k16-apollo-24 -o images --instance-threshold 0.05 --seed-threshold 0.05 --line-width 4 --font-size 0

In [1]:
from openpifpaf.plugins.apollocar3d import constants

constants.CAR_POSE_24

array([[-2.9  ,  4.   , -3.   ],
       [ 2.9  ,  4.   , -3.   ],
       [-2.   ,  2.   , -6.   ],
       [ 2.   ,  2.   , -6.   ],
       [-2.5  ,  0.   , -6.   ],
       [ 2.5  ,  0.   , -6.   ],
       [ 2.6  ,  4.2  ,  0.   ],
       [ 3.2  ,  0.2  , -4.2  ],
       [ 3.   ,  0.3  ,  3.15 ],
       [ 3.1  ,  2.1  ,  2.25 ],
       [ 2.4  ,  4.3  ,  1.575],
       [-2.4  ,  4.3  ,  1.575],
       [ 2.5  ,  2.2  ,  4.5  ],
       [-2.5  ,  2.2  ,  4.5  ],
       [ 2.1  ,  0.1  ,  4.5  ],
       [-2.1  ,  0.1  ,  4.5  ],
       [-2.6  ,  4.2  ,  0.   ],
       [-3.1  ,  2.1  ,  2.25 ],
       [-3.   ,  0.3  ,  3.15 ],
       [-3.2  ,  0.2  , -4.2  ],
       [ 1.   ,  1.3  ,  4.5  ],
       [-1.   ,  1.3  ,  4.5  ],
       [ 2.8  ,  3.   , -2.1  ],
       [-2.8  ,  3.   , -2.1  ]])

In [ ]:
CAR_KEYPOINTS_24 = [
    'front_up_right',       # 1
    'front_up_left',        # 2
    'front_light_right',    # 3
    'front_light_left',     # 4
    'front_low_right',      # 5
    'front_low_left',       # 6
    'central_up_left',      # 7
    'front_wheel_left',     # 8
    'rear_wheel_left',      # 9
    'rear_corner_left',     # 10
    'rear_up_left',         # 11
    'rear_up_right',        # 12
    'rear_light_left',      # 13
    'rear_light_right',     # 14
    'rear_low_left',        # 15
    'rear_low_right',       # 16
    'central_up_right',     # 17
    'rear_corner_right',    # 18
    'rear_wheel_right',     # 19
    'front_wheel_right',    # 20
    'rear_plate_left',      # 21
    'rear_plate_right',     # 22
    'mirror_edge_left',     # 23
    'mirror_edge_right',    # 24
]

In [ ]:

CAR_KEYPOINTS_66 = [
    "top_left_c_left_front_car_light",      # 0
    "bottom_left_c_left_front_car_light",   # 1
    "top_right_c_left_front_car_light",     # 2
    "bottom_right_c_left_front_car_light",  # 3
    "top_right_c_left_front_fog_light",     # 4
    "bottom_right_c_left_front_fog_light",  # 5
    "front_section_left_front_wheel",       # 6
    "center_left_front_wheel",              # 7
    "top_right_c_front_glass",              # 8
    "top_left_c_left_front_door",           # 9
    "bottom_left_c_left_front_door",        # 10
    "top_right_c_left_front_door",          # 11
    "middle_c_left_front_door",             # 12
    "front_c_car_handle_left_front_door",   # 13
    "rear_c_car_handle_left_front_door",    # 14
    "bottom_right_c_left_front_door",       # 15
    "top_right_c_left_rear_door",           # 16
    "front_c_car_handle_left_rear_door",    # 17
    "rear_c_car_handle_left_rear_door",     # 18
    "bottom_right_c_left_rear_door",        # 19
    "center_left_rear_wheel",               # 20
    "rear_section_left_rear_wheel",         # 21
    "top_left_c_left_rear_car_light",       # 22
    "bottom_left_c_left_rear_car_light",    # 23
    "top_left_c_rear_glass",                # 24
    "top_right_c_left_rear_car_light",      # 25
    "bottom_right_c_left_rear_car_light",   # 26
    "bottom_left_c_trunk",                  # 27
    "Left_c_rear_bumper",                   # 28
    "Right_c_rear_bumper",                  # 29
    "bottom_right_c_trunk",                 # 30
    "bottom_left_c_right_rear_car_light",   # 31
    "top_left_c_right_rear_car_light",      # 32
    "top_right_c_rear_glass",               # 33
    "bottom_right_c_right_rear_car_light",  # 34
    "top_right_c_right_rear_car_light",     # 35
    "rear_section_right_rear_wheel",        # 36
    "center_right_rear_wheel",              # 37
    "bottom_left_c_right_rear_car_door",    # 38
    "rear_c_car_handle_right_rear_car_door",    # 39
    "front_c_car_handle_right_rear_car_door",   # 40
    "top_left_c_right_rear_car_door",       # 41
    "bottom_left_c_right_front_car_door",   # 42
    "rear_c_car_handle_right_front_car_door",   # 43
    "front_c_car_handle_right_front_car_door",  # 44
    "middle_c_right_front_car_door",        # 45
    "top_left_c_right_front_car_door",      # 46
    "bottom_right_c_right_front_car_door",  # 47
    "top_right_c_right_front_car_door",     # 48
    "top_left_c_front_glass",               # 49
    "center_right_front_wheel",             # 50
    "front_section_right_front_wheel",      # 51
    "bottom_left_c_right_fog_light",        # 52
    "top_left_c_right_fog_light",           # 53
    "bottom_left_c_right_front_car_light",  # 54
    "top_left_c_right_front_car_light",     # 55
    "bottom_right_c_right_front_car_light",  # 56
    "top_right_c_right_front_car_light",     # 57
    "top_right_c_front_lplate",             # 58
    "top_left_c_front_lplate",              # 59
    "bottom_right_c_front_lplate",           # 60
    "bottom_left_c_front_lplate",          # 61
    "top_left_c_rear_lplate",               # 62
    "top_right_c_rear_lplate",              # 63
    "bottom_right_c_rear_lplate",           # 64
    "bottom_left_c_rear_lplate", ]            # 65
